In [51]:
#Import Dependencies
import pandas as pd
import gmaps
import requests
import json
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [52]:
#Get the dataframe from the csv generated in the WeatherPy exercise
cities_path = "../WeatherPy/Output/cities.csv"
cities_df = pd.read_csv(cities_path)
cities_df

,City_ID,City,Cloudiness(%),Country,Date,Humidity(%),Latitude,Longitude,Max Temp(F),Wind Speed(mph)
0,0,bodden town,40,KY,1613174544,69,19.28,-81.25,80.60,12.66
1,1,khatanga,93,RU,1613174483,86,71.97,102.50,-25.91,7.47
2,2,rikitea,37,PF,1613174486,70,-23.12,-134.97,80.56,9.15
3,3,ponta do sol,15,PT,1613174476,86,32.67,-17.10,55.99,5.99
4,4,hobart,75,AU,1613174377,48,-42.88,147.33,66.00,9.22
...,...,...,...,...,...,...,...,...,...,...
495,495,ca mau,98,VN,1613174109,79,9.18,105.15,74.66,7.14
496,496,dukat,8,RU,1613174654,84,62.55,155.55,-12.60,4.34
497,497,tukrah,27,LY,1613174655,77,32.53,20.58,53.33,1.70
498,498,libenge,9,CD,1613174655,26,3.65,18.63,71.80,3.74


In [53]:
#Create a heat map that displays the humidity for every city

# Store latitude and longitude in locations
locations = cities_df[['Latitude', 'Longitude']]
# Store the humidity values
humidity = cities_df['Humidity(%)'].astype(float)
max_int = cities_df['Humidity(%)'].max()

# Plot Heatmap
fig1 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_int,
                                 point_radius=1)
# Add layer
fig1.add_layer(heat_layer)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))

In [54]:
#Narrow down the DataFrame to find your ideal weather condition.
#Considerations: Max tem: 70F to 95F, Humidity: 30%-60%, Cloudiness <5
ideal_df = cities_df.loc[(cities_df['Max Temp(F)']<95) & (cities_df['Max Temp(F)']>=70) & (cities_df['Humidity(%)']>=30)\
                        & (cities_df['Humidity(%)']<51) & (cities_df['Cloudiness(%)']<5)]

ideal_df


,City_ID,City,Cloudiness(%),Country,Date,Humidity(%),Latitude,Longitude,Max Temp(F),Wind Speed(mph)
93,93,iquique,0,CL,1613174541,44,-20.22,-70.14,77.00,17.27
242,242,belem de sao francisco,1,BR,1613174596,44,-8.75,-38.97,82.58,22.93
476,476,ahipara,0,NZ,1613174208,50,-35.17,173.17,72.82,3.94


In [56]:
# Get the coordinates & setup the variables for the url search
ideal_coor = ideal_df[['City','Country','Latitude', 'Longitude']]
coord = ideal_df[['Latitude', 'Longitude']]
target_search = "lodging"
target_radius = 5000
target_type = "hotel"

label_list = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for city, country, lat, long in ideal_coor.itertuples(index=False):
    target_coordinates = f"{lat}, {long}"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Save the hotel name
    if len(response["results"]) != 0:
        hotel = response["results"][0]["name"]
    else:
        hotel = "No Hotels nearby"
    
    #Get the information for the content box
    label_dict = {
        "hotel": hotel,
        "city": city,
        "country": country
    }
    
    label_list.append(label_dict)

info_box = """
<dl>
<dt>Hotel</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box.format(**entry) for entry in label_list] 

# Plot
fig2 = gmaps.figure()

# Assign the marker layer to a variable

markers = gmaps.marker_layer(coord, info_box_content=hotel_info)
fig2.add_layer(heat_layer)
fig2.add_layer(markers)
fig2

Figure(layout=FigureLayout(height='420px'))